In [1]:
import uproot
import numpy as np
import torch

import GraphBuilder

In [2]:
#############################
# Let's read the file
#############################

file = 'stupidUPROOT_0'
inputFileName = '/Users/isobel/Desktop/DUNE/2024/Hierarchy/HigherTier/files/without2DPFPs/' + file + '.npz'

In [3]:
data = np.load(inputFileName)

#############################
# Get file masks
#############################
particle_mask_main = data['particle_mask']

#############################
# Get event-level stuff
#############################
recoNuVertexX = data['recoNuVertexX']
recoNuVertexY = data['recoNuVertexY']
recoNuVertexZ = data['recoNuVertexZ']

#############################
# True information.. (for cheating)
#############################
pfpTruePDG_main = data['pfpTruePDG']
pfpTrueMomX_main = data['pfpTrueMomX']
pfpTrueMomY_main = data['pfpTrueMomY']
pfpTrueMomZ_main = data['pfpTrueMomZ']
    
#############################
# Get GNN node stuff 
#############################
trackShowerScore_main = data['trackShowerScore']
nHits_main = data['nHits']
charge_main = data['charge']
vertexX_main = data['vertexX']
vertexY_main = data['vertexY']
vertexZ_main = data['vertexZ']
trackEndX_main = data['trackEndX']
trackEndY_main = data['trackEndY']
trackEndZ_main = data['trackEndZ']
showerDirX_main = data['showerDirX']  # not the best direction estimate, placeholder
showerDirY_main = data['showerDirY']
showerDirZ_main = data['showerDirZ']
ivysaurusMuon_main = data['ivysaurusMuon']
ivysaurusProton_main = data['ivysaurusProton']
ivysaurusPion_main = data['ivysaurusPion']
ivysaurusElectron_main = data['ivysaurusElectron']
ivysaurusPhoton_main = data['ivysaurusPhoton']
trackLength_main = data['trackLength']
displacement_main = data['displacement']
dca_main = data['dca']
nuVertexEnergyAsymmetry_main = data['nuVertexEnergyAsymmetry']
nuVertexEnergyWeightedMeanRadialDistance_main = data['nuVertexEnergyWeightedMeanRadialDistance']

#############################
# Get node truth
#############################
trueTrackID_main = data['trueTrackID']
trueVisibleParentTrackID_main = data['trueVisibleParentTrackID']
trueVisibleGeneration_main = data['trueVisibleGeneration']

#############################
# How many entries are we working with?
#############################
nEntries = particle_mask_main.shape[0]
print('We are working with:', nEntries, 'entries')

#############################
# GNN config
#############################

modeDict = {
    "ADD_NEUTRINO"                 : True,
    "CHEAT_DIRECTION"              : True,
    "CHEAT_PID"                    : True, 
    "EDGE_FRACTION"                : 0.8,
    "DO_NORMALISATION"             : True,
    "MAX_NODE_CLASS"               : 4
}

We are working with: 98400 entries


In [4]:
#############################
# Let's create our graphs!
#############################

data_set_pos = []
data_set_neg = []
data_set_FC = []


for iEvent in range(nEntries) : 
    
    ########################################
    # Read our input arrays
    ########################################
    particle_mask_event = particle_mask_main[iEvent]
    
    eventDict = {
        "recoNuVertexX"                            : recoNuVertexX[iEvent], \
        "recoNuVertexY"                            : recoNuVertexY[iEvent], \
        "recoNuVertexZ"                            : recoNuVertexZ[iEvent], \
        "nParticles"                               : np.count_nonzero(particle_mask_event), \
        "trackShowerScore"                         : trackShowerScore_main[iEvent][particle_mask_event], \
        "nHits"                                    : nHits_main[iEvent][particle_mask_event], \
        "charge"                                   : charge_main[iEvent][particle_mask_event], \
        "vertexX"                                  : vertexX_main[iEvent][particle_mask_event], \
        "vertexY"                                  : vertexY_main[iEvent][particle_mask_event], \
        "vertexZ"                                  : vertexZ_main[iEvent][particle_mask_event], \
        "trackEndX"                                : trackEndX_main[iEvent][particle_mask_event], \
        "trackEndY"                                : trackEndY_main[iEvent][particle_mask_event], \
        "trackEndZ"                                : trackEndZ_main[iEvent][particle_mask_event], \
        "showerDirX"                               : showerDirX_main[iEvent][particle_mask_event], \
        "showerDirY"                               : showerDirY_main[iEvent][particle_mask_event], \
        "showerDirZ"                               : showerDirZ_main[iEvent][particle_mask_event], \
        "ivysaurusMuon"                            : ivysaurusMuon_main[iEvent][particle_mask_event], \
        "ivysaurusProton"                          : ivysaurusProton_main[iEvent][particle_mask_event], \
        "ivysaurusPion"                            : ivysaurusPion_main[iEvent][particle_mask_event], \
        "ivysaurusElectron"                        : ivysaurusElectron_main[iEvent][particle_mask_event], \
        "ivysaurusPhoton"                          : ivysaurusPhoton_main[iEvent][particle_mask_event], \
        "trackLength"                              : trackLength_main[iEvent][particle_mask_event], \
        "displacement"                             : displacement_main[iEvent][particle_mask_event], \
        "dca"                                      : dca_main[iEvent][particle_mask_event], \
        "isNeutrinoPDG"                            : np.zeros(pfpTruePDG_main[iEvent][particle_mask_event].shape), \
        "nuVertexEnergyAsymmetry"                  : nuVertexEnergyAsymmetry_main[iEvent][particle_mask_event], \
        "nuVertexEnergyWeightedMeanRadialDistance" : nuVertexEnergyWeightedMeanRadialDistance_main[iEvent][particle_mask_event], \
        "trueTrackID"                              : trueTrackID_main[iEvent][particle_mask_event], \
        "trueVisibleParentTrackID"                 : trueVisibleParentTrackID_main[iEvent][particle_mask_event], \
        "trueMomX"                                 : pfpTrueMomX_main[iEvent][particle_mask_event], \
        "trueMomY"                                 : pfpTrueMomY_main[iEvent][particle_mask_event], \
        "trueMomZ"                                 : pfpTrueMomZ_main[iEvent][particle_mask_event], \
        "truePDG"                                  : pfpTruePDG_main[iEvent][particle_mask_event], \
        "trueVisibleGeneration"                    : trueVisibleGeneration_main[iEvent][particle_mask_event]
    }

        
    data_pos, data_neg, data_FC, pfpIndex = GraphBuilder.EdgeAttributeGraphBuilder(eventDict, modeDict)

    if ((data_pos.num_edges == 0) and (data_neg.num_edges == 0)) :
        continue
        
    data_set_pos.append(data_pos)
    data_set_neg.append(data_neg)
    data_set_FC.append(data_FC)

In [5]:
#############################
# Let's save our graphs!
#############################
outputFileSuffix = '_ParticleToNeutrinoTruth_EdgeAttr_SWAP_0.pt'

torch.save(data_set_pos, str('data_set_pos' + outputFileSuffix))
torch.save(data_set_neg, str('data_set_neg' + outputFileSuffix))
torch.save(data_set_FC, str('data_set_FC' + outputFileSuffix))